In [1]:
from dotenv import load_dotenv # Load langsmith environment variables
load_dotenv()

True

In [2]:
# Import needed libraries and usefull methods
from langchain_ollama import ChatOllama

LLM_MODEL = "llama3.2:3b"

# Prompt Engineering 101

Now that we have a way of utilizing memory, we can start testing with some prompt engineering techniques. There are tons of different techniques, but we'll try to focus on 3 main techniques in order to try and achieve best results, namelly:
1. Zero Shot: Basically what we did on the beginning, were we just ask the LLM without providing any type of context.
2. Functioning Shot: Like the Zero Shot, but we provide some context to the LLM as to what it should respond as (ie: a developer, consultant, doctor, etc). We already made this before when utilizing the system message.
3. Few Shots: We provide the LLM with some extra steps (human interaction) until we finally achieve the final answer we actually need. This is one of the most common techniques when dealing with generic chatbots, and humans are used to utilize this withouth even knowing it.
4. Chain of tought: We provide the LLM with small and discrete tasks, until we achieve the final interation.

There are many more techniques, and you can also utilize a combination of those techniques. Feel free to experiment as you see fit.

Now let's play a little with each technique.

## Zero shot
To utilize a Zero shot technique, we could just ask the model and get the response.

In [3]:
llm_model = ChatOllama(model=LLM_MODEL, temperature=0.3)
response = llm_model.invoke("What are the top 5 programming languages?")
print(response.content)

The ranking of the top programming languages can vary depending on factors such as industry, job requirements, and personal preferences. However, based on various sources, including the TIOBE Index, GitHub, and Stack Overflow, here are five of the most popular programming languages:

1. **JavaScript**: JavaScript is a versatile language that is widely used for web development, mobile app development, and server-side programming. Its popularity has been increasing in recent years due to its widespread adoption in modern web applications.
2. **Python**: Python is a high-level language known for its simplicity, readability, and versatility. It's widely used in data science, machine learning, artificial intelligence, and web development. Its popularity has been growing rapidly in recent years due to its ease of use and extensive libraries.
3. **Java**: Java is an object-oriented language that's widely used for Android app development, web development, and enterprise software development. I

# Functioning shot

To work with the functioning shot, we can either provide the LLM function inside the prompt, or with a system message. We usually make it with system message when we don't have control of the input of the user, this was the previous examples. Since we already worked with the system message, let's try working directly on the prompt.

In [4]:
llm_model = ChatOllama(model=LLM_MODEL, temperature=0.3)
response = llm_model.invoke("You are a seasoned developer that likes to experiment with new languages. What are the top 5 programming languages?")
print(response.content)

As a seasoned developer, I've had the pleasure of working with numerous programming languages over the years. While it's difficult to pinpoint a single "top" language, I can give you an overview of the most popular and in-demand programming languages based on various sources such as GitHub, Stack Overflow, and industry trends.

Here are my top 5 picks:

1. **JavaScript**: JavaScript is the king of client-side scripting and has become the de facto language for web development. Its versatility, dynamic nature, and widespread adoption make it an essential skill for any developer. With frameworks like React, Angular, and Vue.js, JavaScript is used in a wide range of applications, from mobile apps to desktop software.
2. **Python**: Python is a versatile language that has gained immense popularity in recent years due to its simplicity, readability, and extensive libraries. It's widely used in data science, machine learning, web development (with frameworks like Django and Flask), and automa

## Few shot and Chain of tought

We can only work with the Few Shots (and chain of tought) if we have some kind of memory, since we relly on the memory to provide the context to the LLM. Let's try utilizing a simple Langgraph graph, and work with an LLM while providing context for it to answer the final question.

The biggest difference from the Few Shots and Chain of Tought is the actuall content we are providing. We'll try to work with both at the same time in this example, by utilizing the Chain of Tought, as a Few Shots prompting engineering.

In [5]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, END, MessagesState, StateGraph

def create_graph():
    """Create a state graph for the chat with persistence."""    

    # Create our custom node.
    def chat_agent(state: MessagesState):
        messages = state['messages']
        prompt = messages
        chat_with_persistence = ChatOllama(model=LLM_MODEL, temperature=0.7)
        response = chat_with_persistence.invoke(prompt)
        return {'messages':[response]}

    # Create the memory checkpointer
    memory = MemorySaver()

    # Build the state graph
    graph_builder = StateGraph(MessagesState)
    graph_builder.add_node('chat_agent', chat_agent)
    graph_builder.add_edge(START, 'chat_agent')
    graph_builder.add_edge('chat_agent', END)

    graph = graph_builder.compile(checkpointer=memory)

    return graph

In [6]:
thread_id = "cot_math_representation"
config = {'configurable': {'thread_id': thread_id}}
graph = create_graph()

In [7]:
# Some context
user_input = "You are an authomatic math assistant, you will be given some information about math operations, and then you will be asked to solve a math operation. Provide the answer with a brief explanation."
# Add some information
user_input = "We are utilizing the following letters to represent numbers: A, B, C, D, E, F, G, H, I, J, K, L, M, N, O, P, Q, R, S, T, U, V, W, X, Y and Z. The numbers they represent are 1, 2, 3, etc."
graph.invoke({'messages': [{'role': 'user', 'content': user_input}]}, config=config)
user_input = "In that case, what happens when we add A and C?" # 4
graph.invoke({'messages': [{'role': 'user', 'content': user_input}]}, config=config)
user_input = "Then add A and C, and store it in a variable called Delta." # Delta = 4, or D
graph.invoke({'messages': [{'role': 'user', 'content': user_input}]}, config=config)
# Make a question
user_input = "Now remove C from Delta, what is the result in our representation?" # 1, or A
response = graph.invoke({'messages': [{'role': 'user', 'content': user_input}]}, config=config)
print(response['messages'][-1].content) 

We had:
Δ = 4

Now, let's subtract C (which is 3) from Δ:

Δ - 3 = 4 - 3
= 1

So, after removing C from Delta, we're left with A.
